<h1>Capstone Project Notebook<h1>

This jupyter Notebook will be used for the capstone project of the Coursera course "Applied Data Science Capstone

In [1]:
#import libraries
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize # librarie for converting JSON to a DF

from sklearn.cluster import KMeans

#!conda install -c conda-forge geopy 
from geopy.geocoders import Nominatim #librarie for finding the coordinates of a places

In [2]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


<h2>Segmentation and Clustering Neighborhoods in Toronto<h2>

<h3> 1st Part: WebScraping<h3>

In [3]:
#!pip install bs4 #Uncomment if bs4 is not install 


In [4]:
#import libraries for webscreaping
from bs4 import BeautifulSoup
import requests

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #Wikipedia link to Tonronto neighbourhoods
data = requests.get(url).text #converts the html code to a string
soup = BeautifulSoup(data,"html5lib")# Create a soup object with the data value

        

In [6]:
#print(soup.prettify())#Print the HTML in nested structure

The first table is the one with the information we need

In [7]:
table = soup.find_all("table")[0] # this is the first table of the Wiki page

table.find("tr")

<tr>
<td style="width:11%;">
<p>M1A<br/><span style="font-size:85%;">Not assigned</span>
</p>
</td>
<td style="width:11%;">
<p>M2A<br/><span style="font-size:85%;">Not assigned</span>
</p>
</td>
<td style="width:11%;">
<p>M3A<br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td>
<td style="width:11%;">
<p>M4A<br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
</p>
</td>
<td style="width:11%;">
<p>M5A<br/><span style="font-size:85%;"><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a><br/>(<a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a> / <a href="/wiki/Harbourfront,_Toronto" title="Harbourfront, Toronto">Harbourfront</a>)</span>
</p>
</td>
<td style="width:11%;">
<p>M6A<br/><span style="font

We will test the best way to save the information within the the cells of the table

In [8]:
#This code is to learn how to extract the information we need
Dict ={}
single_row = table.find("tr").find_all("td")[8]
print(single_row.text) # this is how text within a cell 
split = single_row.text.replace(")","").replace("\n","").split("(")
print(split)
print(split[0])
palabra = split[0]
print(palabra[0:3])#the first 3 character are the postal code
Dict[palabra[0:3]] =[[],[]]
Dict[palabra[0:3]][0]= palabra[3:]


M9AEtobicoke(Islington Avenue)


['M9AEtobicoke', 'Islington Avenue']
M9AEtobicoke
M9A


In [9]:
def combine(list): 
    #Funtion to combine all the Neighborhood  in that postal code as 
    # single string
    word = ""
    for i in list:
        if word != "":
            word =(word +", "+ i)
        else:
            word = i
    return(word)
        
    

Dict ={}

for row in table.find_all("tr"):
    col = row.find_all("td")
    for box in col:
        dis_neig = box.text.replace(")","").replace("\n","").split("(")
        
        if dis_neig[0][3:] != "Not assigned": #This will avoid the cells with the "Not assigned" with in them
            code_district = dis_neig[0]
            
            negihs = dis_neig[1].split(" / ")# create a list with all the neighborhoods with in the cell
            
                        
            if code_district[0:3] not in Dict.keys():
                Dict[code_district[0:3]] =[[],[]]
                Dict[code_district[0:3]][0] = code_district[3:]
                Dict[code_district[0:3]][1] = combine(negihs)
            
                
                
#Check the result               

print(Dict)

{'M3A': ['North York', 'Parkwoods'], 'M4A': ['North York', 'Victoria Village'], 'M5A': ['Downtown Toronto', 'Regent Park, Harbourfront'], 'M6A': ['North York', 'Lawrence Manor, Lawrence Heights'], 'M7A': ["Queen's Park", 'Ontario Provincial Government'], 'M9A': ['Etobicoke', 'Islington Avenue'], 'M1B': ['Scarborough', 'Malvern, Rouge'], 'M3B': ['North York', 'Don MillsNorth'], 'M4B': ['East York', 'Parkview Hill, Woodbine Gardens'], 'M5B': ['Downtown Toronto', 'Garden District, Ryerson'], 'M6B': ['North York', 'Glencairn'], 'M9B': ['Etobicoke', 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'], 'M1C': ['Scarborough', 'Rouge Hill, Port Union, Highland Creek'], 'M3C': ['North York', 'Don MillsSouth'], 'M4C': ['East York', 'Woodbine Heights'], 'M5C': ['Downtown Toronto', 'St. James Town'], 'M6C': ['York', 'Humewood-Cedarvale'], 'M9C': ['Etobicoke', 'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood'], 'M1E': ['Scarborough', 'Guildwood, Morningside, West

we create a DataFrame the Dictionary

In [10]:

df = pd.DataFrame.from_dict(Dict,orient="index")
df.reset_index(inplace=True)
df.columns= ["PostalCode","Borough","Neighborhood"]
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
df[df["Neighborhood"].isnull()]

,PostalCode,Borough,Neighborhood


In [12]:
print("Shape: ",df.shape)

Shape:  (103, 3)


<h3> 2 Part: Adding the coordinates<h3>

I will use the csv file with coordinates provided in the project instructions  

In [13]:
coor = pd.read_csv("Geospatial_Coordinates.csv")
print("Shape: ", coor.shape)
coor.head()


Shape:  (103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
toronto_data = df.set_index("PostalCode").join(coor.set_index("Postal Code")) #Merged the DF with the coordinates with the DF extracted from the Wiki
toronto_data.reset_index(inplace=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [15]:
#Check if the merge was done correctly
print(toronto_data[toronto_data["PostalCode"]=="M1B"])
toronto_data.head()


  PostalCode      Borough    Neighborhood   Latitude  Longitude
6        M1B  Scarborough  Malvern, Rouge  43.806686 -79.194353


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Now we check for None values

In [16]:
toronto_data[toronto_data["Latitude"].isnull()]#Check for null values

,PostalCode,Borough,Neighborhood,Latitude,Longitude


<h4>3rd Part: Clustering<h4>

In [17]:
CLIENT_ID = 'XZ4CXTKCANOCIZW3XE50X0HGGKAB4G3ZOLL5MVJLW5PQRS25' # your Foursquare ID
CLIENT_SECRET = '5ZBUG2NR321W3HLRTKHB5KR052PVDPKV5ES4B4FMXCD3BPBG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [18]:
#TEst code
neighborhood_latitude =toronto_data.iloc[0]["Latitude"]
radius = 500
neighborhood_longitude = toronto_data.iloc[0]["Longitude"]
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=XZ4CXTKCANOCIZW3XE50X0HGGKAB4G3ZOLL5MVJLW5PQRS25&client_secret=5ZBUG2NR321W3HLRTKHB5KR052PVDPKV5ES4B4FMXCD3BPBG&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [19]:
#Test Code
result = requests.get(url).json()
result["response"]['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
   'name': 'Brookbanks Park',
   'location': {'address': 'Toronto',
    'lat': 43.751976046055574,
    'lng': -79.33214044722958,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.751976046055574,
      'lng': -79.33214044722958}],
    'distance': 245,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']},
   'categories': [{'id': '4bf58dd8d48988d163941735',
     'name': 'Park',
     'pluralName': 'Parks',
     'shortName': 'Park',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []},
   'venuePage': {'id': '600917367'}},
  'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0'},
 {'r

In [20]:
#Function that extracts the category of the  venue ( it´s takes a DF row)
def get_category_type(row):
    try:
        categories_list = row['categories'] #this list will be a lot of info aboute the venue categorie
    except:
        categories_list =row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']
    
    

In [21]:
venues = result['response']['groups'][0]['items']
venues

nearby_venues = json_normalize(venues) #This is a DataFrame with the nearby venues

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]#Creates a DF with the rows above

nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]


C:\Users\Julio\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


In [22]:
nearby_venues

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,KFC,Fast Food Restaurant,43.754387,-79.333021
2,TTC stop #8380,Bus Stop,43.752672,-79.326351
3,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes): # this will check all the neighborhoods in names
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)     

In [24]:
toronto_venues = getNearbyVenues(toronto_data["Neighborhood"],toronto_data["Latitude"],toronto_data["Longitude"])

In [25]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
3,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
...,...,...,...,...,...,...,...
2140,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,McDonald's,43.630007,-79.518041,Fast Food Restaurant
2141,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Jim & Maria's No Frills,43.631152,-79.518617,Grocery Store
2142,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
2143,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym


In [26]:
toronto_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
WillowdaleWest,6,6,6,6,6,6
Woburn,4,4,4,4,4,4
Woodbine Heights,8,8,8,8,8,8


In [27]:
print("There ara {} uniques categories.".format(len(toronto_venues["Venue Category"].unique())))

There ara 275 uniques categories.


In [28]:
# one hot encoding

toronto_onehot = pd.get_dummies(toronto_venues[["Venue Category"]],prefix="",prefix_sep="")
toronto_onehot.drop(columns="Neighborhood",inplace =True)
toronto_onehot["xNeighborhood"]=toronto_venues["Neighborhood"]



fixed_columns= [toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
fixed_columns
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.rename(columns={"xNeighborhood":"Neighborhood"},inplace=True)

toronto_onehot

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2141,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2142,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2143,"Mimico NW, The Queensway West, South of Bloor,...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
toronto_grouped = toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,WillowdaleWest,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
toronto_grouped.shape

(101, 275)

In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index() 
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]# the firs row will be Neighborhood| hood
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt ----
                       venue  freq
0             Clothing Store   0.2
1             Breakfast Spot   0.2
2                     Lounge   0.2
3  Latin American Restaurant   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.29
1           Gym  0.14
2  Skating Rink  0.14
3           Pub  0.14
4   Coffee Shop  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.09
1                Coffee Shop  0.09
2                   Pharmacy  0.05
3                 Restaurant  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0       Italian Restaurant  0.08
1              Coffee Shop  

                    venue  freq
0          Clothing Store  0.25
1       Accessories Store  0.12
2             Event Space  0.12
3                Boutique  0.12
4  Furniture / Home Store  0.12


----Lawrence Park----
                venue  freq
0         Swim School  0.25
1  Dim Sum Restaurant  0.25
2                Park  0.25
3            Bus Line  0.25
4   Accessories Store  0.00


----Leaside----
                 venue  freq
0          Coffee Shop  0.09
1  Sporting Goods Shop  0.09
2         Burger Joint  0.06
3     Sushi Restaurant  0.06
4                 Bank  0.06


----Little Portugal, Trinity----
                   venue  freq
0                    Bar  0.12
1            Men's Store  0.05
2  Vietnamese Restaurant  0.05
3             Restaurant  0.05
4            Coffee Shop  0.05


----Malvern, Rouge----
                       venue  freq
0       Fast Food Restaurant   1.0
1          Accessories Store   0.0
2  Middle Eastern Restaurant   0.0
3        Moroccan Restaurant   0.0
4  

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:] #The firs cell is the Neighborhood name
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted["Neighborhood"] = toronto_grouped["Neighborhood"]

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:]= return_most_common_venues(toronto_grouped.iloc[ind,:],num_top_venues)
neighborhoods_venues_sorted.head()   

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Clothing Store,Skating Rink,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Pub,Coffee Shop,Skating Rink,Sandwich Place,Gym,Eastern European Restaurant,Dumpling Restaurant,Electronics Store,Drugstore
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Chinese Restaurant,Bridal Shop,Shopping Mall,Diner,Sandwich Place,Supermarket,Middle Eastern Restaurant
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Toy / Game Store,Hobby Shop,Indian Restaurant,Sushi Restaurant,Café,Butcher,Pub


<h3>Clustering<h3>

In [34]:
#Set number of clusters
kclusters = 5


toronto_grouped_clustering = toronto_grouped.drop("Neighborhood",1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

#toronto_grouped_clustering

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 4])

In [35]:
neighborhoods_venues_sorted.insert(0,"Cluster Labels", kmeans.labels_)

toronto_merged = toronto_data


toronto_merged =toronto_merged.join(neighborhoods_venues_sorted.set_index("Neighborhood"),on= "Neighborhood")

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Fast Food Restaurant,Park,Bus Stop,Food & Drink Shop,Escape Room,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Intersection,French Restaurant,Portuguese Restaurant,Hockey Arena,Coffee Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Bakery,Park,Pub,Theater,Breakfast Spot,Café,Restaurant,French Restaurant,Electronics Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Vietnamese Restaurant,Furniture / Home Store,Coffee Shop,Accessories Store,Boutique,Event Space,Escape Room,Ethiopian Restaurant,Electronics Store
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1.0,Coffee Shop,Sushi Restaurant,Yoga Studio,College Auditorium,Beer Bar,Spa,Smoothie Shop,Burrito Place,Sandwich Place,Café


In [36]:
null_val = toronto_merged[toronto_merged["Cluster Labels"].isnull()]
print(null_val.index)

null_val

Int64Index([5, 95], dtype='int64')


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
toronto_merged.drop(index= null_val.index,inplace =True)
                    

In [38]:

toronto_merged = toronto_merged.reset_index().drop(columns="index")

In [39]:
toronto_merged.shape

(101, 16)

In [40]:
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int)
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4,Fast Food Restaurant,Park,Bus Stop,Food & Drink Shop,Escape Room,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Intersection,French Restaurant,Portuguese Restaurant,Hockey Arena,Coffee Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Theater,Breakfast Spot,Café,Restaurant,French Restaurant,Electronics Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Vietnamese Restaurant,Furniture / Home Store,Coffee Shop,Accessories Store,Boutique,Event Space,Escape Room,Ethiopian Restaurant,Electronics Store
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,College Auditorium,Beer Bar,Spa,Smoothie Shop,Burrito Place,Sandwich Place,Café
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,1,Pool,River,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
97,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Fast Food Restaurant,Hotel,Mediterranean Restaurant,Men's Store,Pub
98,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558,1,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Light Rail Station,Farmers Market,Comic Shop,Park,Recording Studio
99,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,2,Baseball Field,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner


In [41]:
address ="Toronto,Ontario"
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [42]:
#!pip install folium

In [43]:
#Create a map
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

#set color fot scheme
x =np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon,poi,cluster in zip(toronto_merged["Latitude"],toronto_merged["Longitude"],toronto_merged["Neighborhood"],toronto_merged["Cluster Labels"]):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

I try to use up to 8 clusters, but it didn’t seem to give more information of the type of neighborhoods Toronto has. It´s looks to me like Toronto have 2 mayor type of neighborhoods based on the venues nearby. “Cluster 2” has more coffee, shops and general “City stuff”. “Cluster 5” in the other hand has more parks( this makes sense when we look at the map an see that many of the points of the cluster are up the city center, away of the “concrete jungle”) and places to eat. I believed this information will help someone how likes parks very much(or someone how likes his cup coffee every day) were to move

In [44]:
columns = [1,2]+list(range(5,toronto_merged.shape[1]))

def ClusterDF(label):
    return(toronto_merged.loc[toronto_merged["Cluster Labels"]== label,toronto_merged.columns[columns]])

#### Cluster 1

I will call this Cluster the "Park-Yoga-Dumpling" they share all their top 10 Venues

In [45]:
ClusterDF(0)

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,North York,"York Mills, Silver Hills",0,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
51,North York,"Willowdale, Newtonbrook",0,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


#### Cluster 2

This one will be call the Coffe-cluster. This is the bulk of the locations. Coffe seems to be predominant in their top 10 venues

In [46]:
ClusterDF(1)

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,Victoria Village,1,Intersection,French Restaurant,Portuguese Restaurant,Hockey Arena,Coffee Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,Downtown Toronto,"Regent Park, Harbourfront",1,Coffee Shop,Bakery,Park,Pub,Theater,Breakfast Spot,Café,Restaurant,French Restaurant,Electronics Store
3,North York,"Lawrence Manor, Lawrence Heights",1,Clothing Store,Vietnamese Restaurant,Furniture / Home Store,Coffee Shop,Accessories Store,Boutique,Event Space,Escape Room,Ethiopian Restaurant,Electronics Store
4,Queen's Park,Ontario Provincial Government,1,Coffee Shop,Sushi Restaurant,Yoga Studio,College Auditorium,Beer Bar,Spa,Smoothie Shop,Burrito Place,Sandwich Place,Café
5,Scarborough,"Malvern, Rouge",1,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Stadium
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Downtown Toronto,"First Canadian Place, Underground city",1,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Gym,Asian Restaurant,Steakhouse,Salad Place,Bakery
96,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",1,Pool,River,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
97,Downtown Toronto,Church and Wellesley,1,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Fast Food Restaurant,Hotel,Mediterranean Restaurant,Men's Store,Pub
98,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,1,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Light Rail Station,Farmers Market,Comic Shop,Park,Recording Studio


### Cluster 3

I will call this Cluster the yoga-Baseball cluster. They share almost every venue

In [47]:
ClusterDF(2)

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,North York,DownsviewCentral,2,Food Truck,Baseball Field,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio,Diner
56,North York,"Humberlea, Emery",2,Baseball Field,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner
99,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",2,Baseball Field,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner


### Cluster 4

This lonely cluster seems to share some similarities with cluster 1 and 3

In [48]:
ClusterDF(3)

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,York,Weston,3,Convenience Store,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


### Cluster 5

This will be call teh Park-Restaurant cluster. This one seems to have many parks and restaurants.

In [49]:
ClusterDF(4)

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,4,Fast Food Restaurant,Park,Bus Stop,Food & Drink Shop,Escape Room,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant
9,North York,Glencairn,4,Playground,Japanese Restaurant,Park,Bakery,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
15,York,Humewood-Cedarvale,4,Field,Park,Hockey Arena,Trail,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
20,York,Caledonia-Fairbanks,4,Park,Women's Store,Pool,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
34,East YorkEast Toronto,The Danforth East,4,Park,Coffee Shop,Convenience Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
39,North York,DownsviewEast,4,Airport,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
48,North York,"North Park, Maple Leaf Park, Upwood Park",4,Park,Bakery,Construction & Landscaping,Trail,Basketball Court,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
60,Central Toronto,Lawrence Park,4,Park,Bus Line,Swim School,Dim Sum Restaurant,Yoga Studio,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
65,North York,York Mills West,4,Park,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
76,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",4,Sandwich Place,Park,Mobile Phone Shop,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
